## rfm score users

In [12]:
import pandas as pd

# Load the Excel file

df = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/rfm.xlsx')


In [13]:
## rfm scores for users

import datetime as dt

# Calculate RFM scores ( THE DAY OF DATA WAS 2024-07-09)
current_date = dt.datetime(2024, 7, 9)
df['last_buy'] = pd.to_datetime(df['last_buy'], errors='coerce')

# Calculate Recency, Frequency, and Monetary values
df['Recency'] = (current_date - df['last_buy']).dt.days

# Frequency is now represented by average_interval_days
##df['Frequency'] = df['total purchase']
df['Frequency'] = df['average_interval_days']

df['Monetary'] = df['total purchase']

# RFM score calculation
df['R_Score'] = pd.qcut(df['Recency'], 4, labels=[4, 3, 2, 1])
df['F_Score'] = pd.qcut(df['Frequency'], 4, labels=[4, 3, 2, 1])
df['M_Score'] = pd.qcut(df['Monetary'].rank(method='first'), 4, labels=[1, 2, 3, 4])

# Combine R, F, M scores into a single RFM score
# df['RF'] =df['R_Score'].astype(int) + df['F_Score'].astype(int) 

df['RFM_Score'] = df['R_Score'].astype(str) + df['F_Score'].astype(str) + df['M_Score'].astype(str)

# Define RFM segments

rfm_segments = {
    '444': 'Best Customers',
    '344': 'Loyal Customers',
    '244': 'Big Spenders',
    '144': 'Almost Lost',
    '434': 'At Risk',
    '334': 'Need Attention',
    '234': 'Lost Customers',
    '134': 'Hibernating',
}

df['Segment'] = df['RFM_Score'].map(rfm_segments).fillna('Others')

df.to_excel('rfm_complete.xlsx')


## rfm score stations ( check if freq defined with 1/average_interval_days would affect the results)

In [36]:
import pandas as pd
import datetime as dt

# Load the data
df = pd.read_excel(r'C:\Users\Sasan\Desktop\tasks\user signup and wallet - new/rfm_station.xlsx')

# Define the current date
current_date = dt.datetime(2024, 7, 9)
df['last_buy'] = pd.to_datetime(df['last_buy'], errors='coerce')

# Calculate Recency, Frequency, and Monetary values
df['Recency'] = (current_date - df['last_buy']).dt.days
df['Frequency'] = df['average_interval_days']
df['Monetary'] = df['total purchase']

# Define thresholds for the categories
recency_thresholds = [0.0, 29.24]
frequency_thresholds = [0.05, 0.10]
monetary_thresholds = [1033.52, 2765.35]

# Function to categorize based on thresholds
def categorize(value, thresholds):
    if value <= thresholds[0]:
        return 3
    elif value <= thresholds[1]:
        return 2
    else:
        return 1

# Apply categorization
df['R_Score'] = df['Recency'].apply(lambda x: categorize(x, recency_thresholds))
df['F_Score'] = df['Frequency'].apply(lambda x: categorize(x, frequency_thresholds))
df['M_Score'] = df['Monetary'].apply(lambda x: categorize(x, monetary_thresholds))

# Combine R, F, M scores into a single RFM score
df['RFM_Score'] = df['R_Score'].astype(str) + df['F_Score'].astype(str) + df['M_Score'].astype(str)

# Define RFM segments based on new scoring
rfm_segments = {
    '333': 'Best Customers',
    '233': 'Loyal Customers',
    '133': 'Big Spenders',
    '313': 'Almost Lost',
    '323': 'At Risk',
    '223': 'Need Attention',
    '123': 'Lost Customers',
    '213': 'Hibernating',
}

df['Segment'] = df['RFM_Score'].map(rfm_segments).fillna('Others')

# Save the result to an Excel file
df.to_excel('rfm_station_complete.xlsx', index=False)

## other visualization and determining fave irems for low and high profile customers

In [ ]:
# RFM Segments Distribution
import matplotlib.pyplot as plt


segment_distribution = df['Segment'].value_counts()

plt.figure(figsize=(10, 6))
segment_distribution.plot(kind='bar', color='skyblue')
plt.title('RFM Segments Distribution')
plt.xlabel('Segment')
plt.ylabel('Number of Customers')
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Payment Method Distribution
payment_methods = df[['pose_purchase', 'creditcard_purchase', 'app_purchase']].sum()

plt.figure(figsize=(10, 6))
payment_methods.plot(kind='bar', color='mediumpurple')
plt.title('Payment Method Distribution')
plt.xlabel('Payment Method')
plt.ylabel('Number of Purchases')
plt.xticks(rotation=0)
plt.show()


In [ ]:
# It appears that the columns 'item_name' and 'station_name' are not present in the high_value_customers dataframe.
# Let's include these columns from the original data for the high-value customers.

# Merge with the original data to include the 'item_name' and 'station_name' columns
high_value_customers_detailed = high_value_customers.merge(data[['unique_name', 'item_name', 'station_name']], on='unique_name', how='left')

# Group by favorite item and count the occurrences
favorite_item_counts = high_value_customers_detailed['item_name'].value_counts()

# Group by station name and count the occurrences
station_name_counts = high_value_customers_detailed['station_name'].value_counts()

# Calculate average spend (monetary) and frequency for high-value customers
average_spend = high_value_customers['Monetary'].mean()
average_frequency = high_value_customers['Frequency'].mean()

# Compare with overall customer data
overall_average_spend = rfm_data['Monetary'].mean()
overall_average_frequency = rfm_data['Frequency'].mean()

# Prepare the results
purchase_patterns = {
    'Favorite Item Counts': favorite_item_counts,
    'Station Name Counts': station_name_counts,
    'Average Spend (High Value)': average_spend,
    'Average Frequency (High Value)': average_frequency,
    'Overall Average Spend': overall_average_spend,
    'Overall Average Frequency': overall_average_frequency
}

purchase_patterns


In [ ]:
# Define low-value customers (R_Score <= 2, F_Score <= 2, M_Score <= 2)
low_value_customers = rfm_data[(rfm_data['R_Score'].astype(int) <= 2) &
                               (rfm_data['F_Score'].astype(int) <= 2) &
                               (rfm_data['M_Score'].astype(int) <= 2)]

# Merge with the original data to include the 'item_name' and 'station_name' columns for analysis
low_value_customers_detailed = low_value_customers.merge(data[['unique_name', 'item_name', 'station_name']], on='unique_name', how='left')

# Analyze purchase patterns of low-value customers
# Group by favorite item and count the occurrences
low_value_favorite_item_counts = low_value_customers_detailed['item_name'].value_counts()

# Group by station name and count the occurrences
low_value_station_name_counts = low_value_customers_detailed['station_name'].value_counts()

# Calculate average spend (monetary) and frequency for low-value customers
low_value_average_spend = low_value_customers['Monetary'].mean()
low_value_average_frequency = low_value_customers['Frequency'].mean()

# Prepare the results
low_value_purchase_patterns = {
    'Favorite Item Counts': low_value_favorite_item_counts,
    'Station Name Counts': low_value_station_name_counts,
    'Average Spend (Low Value)': low_value_average_spend,
    'Average Frequency (Low Value)': low_value_average_frequency
}

low_value_purchase_patterns


In [6]:
## rfm for stations